<div style="text-align:center; font-family: 'Arial', sans-serif; margin-top: 30px; background-color:#f5f7fa; padding:25px; border-radius:15px; box-shadow: 0 4px 10px rgba(0,0,0,0.1);">
    <h1 style="color:#2E8B57; font-size: 50px; margin-bottom: 10px;">Plant Disease Classifier</h1>
    <p style="color:#555; font-size: 20px; margin-bottom:5px;">by <strong>Seif Eldeen</strong></p>
    <p style="color:#333; font-size: 18px; max-width: 850px; margin:auto; line-height:1.5;">
        This project demonstrates a deep learning model based on <strong>Transfer Learning</strong> using architectures such as   <strong>MobileNetV3</strong> to classify plant leaf images into <strong>15 different disease categories</strong>.
        The dataset includes healthy and infected leaves from multiple plant species. The model was trained with advanced augmentation, early stopping, and fine-tuning to achieve high accuracy while preventing overfitting.
    </p>
    <hr style="width:50%; margin:auto; border:2px solid #2E8B57; margin-top:20px;">
</div>


<div style="background-color:#2E8B57; color:white; padding:10px 20px; border-radius:8px; font-family: 'Arial', sans-serif; margin-top:20px; margin-bottom:10px; box-shadow: 0 3px 6px rgba(0,0,0,0.1);">
    <h2 style="margin:0; font-size:24px;">1) Importing Libraries</h2>
    <p style="color:white; font-size:16px; margin-top:5px; line-height:1.4;">
        In this section, we import all the essential Python libraries for image preprocessing, visualization, and building our deep learning model using transfer learning for plant disease classification.
    </p>
</div>


In [ ]:
!pip install split-folders

In [ ]:
import numpy as np
import os
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from PIL import Image
import random
import splitfolders
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV3Large
from tensorflow.keras.applications.mobilenet_v3 import preprocess_input
from tensorflow.keras import layers, models
from tensorflow.keras.layers import Dense,GlobalAveragePooling2D,Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
import seaborn as sns
from sklearn.metrics import confusion_matrix
from tensorflow.keras.models import load_model
from sklearn.metrics import confusion_matrix, classification_report
import tensorflow as tf
from keras.utils import load_img, img_to_array

import warnings
warnings.filterwarnings('ignore')

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

<div style="background-color:#2E8B57; color:white; padding:10px 20px; border-radius:8px; font-family: 'Arial', sans-serif; margin-top:20px; margin-bottom:10px; box-shadow: 0 3px 6px rgba(0,0,0,0.1);">
    <h2 style="margin:0; font-size:24px;">2) Defining Constants</h2>
    <p style="color:white; font-size:16px; margin-top:5px; line-height:1.4;">
        Here, we define important constants such as dataset paths, image size, batch size, and other parameters used throughout the training process to ensure consistency and easy configuration.
    </p>
</div>


In [ ]:
IMAGES_FOLDERS_PATH = '/kaggle/input/plantdisease/PlantVillage'
OUTPUT_FOLDER = 'plantdisease_split'
BATCH_SIZE = 64
WIDTH = 224
HEIGHT = 224
CHANNELS = 3
EPOCHS = 100

<div style="background-color:#2E8B57; color:white; padding:10px 20px; border-radius:8px; font-family: 'Arial', sans-serif; margin-top:20px; margin-bottom:10px; box-shadow: 0 3px 6px rgba(0,0,0,0.1);">
    <h2 style="margin:0; font-size:24px;">3) Dataset Analysis</h2>
    <p style="color:white; font-size:16px; margin-top:5px; line-height:1.4;">
        In this section, we perform an initial analysis of the dataset by visualizing sample images from each plant disease category.
        The function <strong>VisualizeClasses()</strong> dynamically loads and displays one example from each folder, showing the class name and image resolution to verify the dataset structure and consistency.
        Additionally, a random image is displayed alongside its resized version (224×224) to ensure correct preprocessing before training.
    </p>
</div>


In [ ]:
def VisualizeClasses(folders_path):
    # Get all class folders and sort them
    ClassesNames = os.listdir(folders_path)
    ClassesNames.sort(key=lambda x: (x.split('__')[0], x))
    
    # Display the total number of classes
    NUM_CLASSES = len(ClassesNames)
    print(f"Classes Number: {NUM_CLASSES}")
    
    # Prepare the grid for showing sample images
    cols = 4
    rows = (NUM_CLASSES + cols - 1) // cols
    fig, axes = plt.subplots(rows, cols, figsize=(16, rows * 3))
    axes = axes.flatten()
    
    # Show one sample image from each class
    for i, cat in enumerate(ClassesNames):
        ClassFolderPath = os.path.join(folders_path,cat)
        ImagesName = os.listdir(ClassFolderPath)
        img_path = os.path.join(ClassFolderPath,ImagesName[0])
        with Image.open(img_path) as img:
            width, height = img.size
        image = mpimg.imread(img_path)
        axes[i].imshow(image)
        axes[i].set_title(f"{cat}\n{width}x{height}", fontsize=10)
        axes[i].axis('off')
        
    # Hide unused subplots if any
    for j in range(i + 1, len(axes)):
        axes[j].axis('off')

# Call the function to visualize all classes
VisualizeClasses(IMAGES_FOLDERS_PATH)   



In [ ]:
# Pick a random class folder
ClassesNames = os.listdir(IMAGES_FOLDERS_PATH)
random_class = random.choice(ClassesNames)
class_path = os.path.join(IMAGES_FOLDERS_PATH, random_class)

# Pick a random image from that class
images = os.listdir(class_path)
random_image = random.choice(images)
img_path = os.path.join(class_path, random_image)

# Open and resize the selected image to 224x224
with Image.open(img_path) as img:
    resized_img = img.resize((WIDTH, HEIGHT)) 

# Display the original and resized images side by side
fig, axes = plt.subplots(1, 2, figsize=(8, 4))
axes[0].imshow(mpimg.imread(img_path))
axes[0].set_title("Original")
axes[0].axis('off')

axes[1].imshow(resized_img)
axes[1].set_title(f"Resized ({WIDTH}x{HEIGHT})")
axes[1].axis('off')

# Adjust layout and show the figure
plt.tight_layout()
plt.show()


<div style="background-color:#2E8B57; color:white; padding:10px 20px; border-radius:8px; font-family: 'Arial', sans-serif; margin-top:20px; margin-bottom:10px; box-shadow: 0 3px 6px rgba(0,0,0,0.1);">
    <h2 style="margin:0; font-size:24px;">4) Data Preparation</h2>
    <p style="color:white; font-size:16px; margin-top:5px; line-height:1.4;">
        In this section, the dataset is split into <strong>training (80%)</strong> and <strong>validation (20%)</strong> sets using <strong>SplitFolders</strong>.
        To improve the model's generalization and prevent overfitting, <strong>data augmentation</strong> techniques such as rotation, zooming, shearing, and horizontal flipping are applied using the <strong>ImageDataGenerator</strong> class.
        All images are resized to <strong>224×224</strong> pixels and normalized through <strong>MobileNetV3-Large’s preprocessing function</strong>.
    </p>
</div>


In [ ]:
# Split the dataset into training (80%) and validation (20%) folders
splitfolders.ratio(IMAGES_FOLDERS_PATH, output=OUTPUT_FOLDER, seed=42, ratio=(0.8, 0.2))


In [ ]:
# Define the training and validation dataset paths
TRAIN_FOLDER_PATH = '/kaggle/working/plantdisease_split/train'
VAL_FOLDER_PATH = '/kaggle/working/plantdisease_split/val'

# Data augmentation for training images (adds variations to improve generalization)
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input, # Apply MobileNetV3 preprocessing
    rotation_range=20, # Random rotations
    width_shift_range=0.2, # Horizontal shifts
    height_shift_range=0.2, # Vertical shifts
    shear_range=0.2, # Shear transformations
    zoom_range=0.2,  # Random zoom
    horizontal_flip=True,  # Flip images horizontally
    fill_mode='nearest' # Fill missing pixels after transformations
)
train_image_generator = train_datagen.flow_from_directory(
                                            TRAIN_FOLDER_PATH,
                                            target_size=(WIDTH, HEIGHT), # Resize all images to 224x224
                                            batch_size= BATCH_SIZE, # Number of images per batch
                                            class_mode='categorical') # Multi-class classification


# Validation images (no augmentation, only preprocessing)
val_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
val_image_generator = val_datagen.flow_from_directory(
                                            VAL_FOLDER_PATH,
                                            target_size=(WIDTH, HEIGHT),
                                            batch_size= BATCH_SIZE,
                                            class_mode='categorical')





In [ ]:
class_names = list(val_image_generator.class_indices.keys()) 
class_names

<div style="background-color:#2E8B57; color:white; padding:10px 20px; border-radius:8px; font-family: 'Arial', sans-serif; margin-top:20px; margin-bottom:10px; box-shadow: 0 3px 6px rgba(0,0,0,0.1);">
    <h2 style="margin:0; font-size:24px;">5) Model Building</h2>
    <p style="color:white; font-size:16px; margin-top:5px; line-height:1.4;">
        In this step, the <strong>MobileNetV3-Large</strong> architecture is used as the base model with pretrained <strong>ImageNet</strong> weights.
        The base model’s layers are frozen to retain the learned features during initial training.
        A <strong>Global Average Pooling</strong> layer is added to reduce spatial dimensions, followed by a <strong>Dense</strong> layer with 128 neurons and <strong>ReLU</strong> activation for deeper feature learning.
        A <strong>Dropout</strong> layer with a rate of 0.3 helps prevent overfitting.
        Finally, a <strong>Softmax</strong> output layer classifies images into the <strong>15 plant disease categories</strong>.
    </p>
</div>


In [ ]:
# Load the MobileNetV3-Large model pre-trained on ImageNet (without the top classification layer)
base_model = MobileNetV3Large(weights='imagenet', include_top=False, input_shape=(WIDTH, HEIGHT, CHANNELS))

# Freeze the base model layers to keep pretrained weights during initial training
base_model.trainable = False

# Build the new model on top of the base model
model = Sequential()
model.add(base_model)
model.add(GlobalAveragePooling2D())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(train_image_generator.num_classes,activation='softmax')) #Output layer for classification

model.summary()


<div style="background-color:#2E8B57; color:white; padding:10px 20px; border-radius:8px; font-family: 'Arial', sans-serif; margin-top:20px; margin-bottom:10px; box-shadow: 0 3px 6px rgba(0,0,0,0.1);">
    <h2 style="margin:0; font-size:24px;">6) Model Training (Without Fine-Tuning)</h2>
    <p style="color:white; font-size:16px; margin-top:5px; line-height:1.4;">
        In this phase, the model is trained using the <strong>MobileNetV3-Large</strong> base with frozen layers to retain pretrained ImageNet features.
        The optimizer <strong>Adam</strong> is used with a learning rate of <strong>1e-4</strong>, and the loss function is <strong>categorical crossentropy</strong> for multi-class classification.
        An <strong>EarlyStopping</strong> callback is applied to automatically stop training if no improvement in validation loss is observed for 5 consecutive epochs, restoring the best model weights.
        After training, the model is saved as <strong>"mobilenetv3_before_finetuning.keras"


In [ ]:
early_stopping = keras.callbacks.EarlyStopping(
    patience=5,              # Stop if no improvement for 5 epochs
    restore_best_weights=True # Restore the model weights from the epoch with the best validation loss
)

# Compile the model
model.compile(optimizer=Adam(learning_rate=1e-4), loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
hist = model.fit(
    train_image_generator,
    epochs = EPOCHS,
    verbose = 1,
    callbacks = early_stopping,
    validation_data = val_image_generator
)

In [ ]:
# Save the trained model before applying fine-tuning
model.save("mobilenetv3_before_finetuning.keras")

<div style="background-color:#2E8B57; color:white; padding:10px 20px; border-radius:8px; font-family: 'Arial', sans-serif; margin-top:20px; margin-bottom:10px; box-shadow: 0 3px 6px rgba(0,0,0,0.1);">
    <h2 style="margin:0; font-size:24px;">7) Model Training (With Fine-Tuning)</h2>
    <p style="color:white; font-size:16px; margin-top:5px; line-height:1.4;">
        After completing the initial training phase, fine-tuning is performed by <strong>unfreezing the last 20 layers</strong> of the <strong>MobileNetV3-Large</strong> base model.
        This allows the network to slightly adjust its higher-level feature representations, improving adaptation to the plant disease dataset.
        The optimizer <strong>Adam</strong> is recompiled with a lower learning rate of <strong>1e-5</strong> to ensure stable and controlled weight updates during fine-tuning.
        The model is trained for <strong>10 additional epochs</strong> and saved as <strong>"mobilenetv3_after_finetuning.keras"</strong> for later comparison with the pre-fine-tuned version.
    </p>
</div>


In [ ]:
# Unfreeze the last 30 layers of the base model to fine-tune higher-level features
for layer in base_model.layers[-30:]:
    layer.trainable = True

# Recompile the model with a smaller learning rate for fine-tuning
model.compile(optimizer=Adam(learning_rate=1e-5),  
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train (fine-tune) the model again on the training data
fine_tune_history = model.fit(
    train_image_generator,
    validation_data=val_image_generator,
    epochs=EPOCHS,
    callbacks = early_stopping,
    verbose=1
)

In [ ]:
# Save the trained model after applying fine-tuning
model.save("mobilenetv3_after_finetuning.keras")


<div style="background-color:#2E8B57; color:white; padding:10px 20px; border-radius:8px; font-family: 'Arial', sans-serif; margin-top:20px; margin-bottom:10px; box-shadow: 0 3px 6px rgba(0,0,0,0.1);">
    <h2 style="margin:0; font-size:24px;">8) Model Evaluation & Performance Analysis</h2>
    <p style="color:white; font-size:16px; margin-top:5px; line-height:1.4;">
        After completing both training phases (before and after fine-tuning), the models were thoroughly evaluated to measure performance and compare improvements.
        The evaluation includes three main components:
    </p>
    <ul style="font-size:16px; line-height:1.5; color:white; margin-left:15px;">
        <li><strong>Accuracy Comparison</strong> — Measuring accuracy for each model <strong>before and after fine-tuning</strong> to visualize the improvement in prediction performance.</li>
        <li><strong>Confusion Matrix</strong> — Generated for both model versions to analyze classification behavior, identify misclassified disease categories, and evaluate class-level performance.</li>
        <li><strong>Training Curves (Loss & Accuracy)</strong> — Showing <strong>Training vs Validation Loss and Accuracy</strong> for each model, helping to assess overfitting, underfitting, and general learning stability.</li>
    </ul>
    <p style="color:white; font-size:16px; margin-top:5px; line-height:1.4;">
        These evaluation metrics provide a clear understanding of how fine-tuning impacted the model’s learning quality and its ability to generalize on unseen plant disease images.
    </p>
</div>


In [ ]:
model_before = load_model("mobilenetv3_before_finetuning.keras")
loss, accuracy = model_before.evaluate(val_image_generator, verbose=1)
print(f"Test Accuracy: {accuracy * 100:.2f}%")
print(f"Test Loss: {loss:.4f}")


In [ ]:
model_after = load_model("mobilenetv3_after_finetuning.keras")
loss, accuracy = model_after.evaluate(val_image_generator, verbose=1)
print(f"Test Accuracy: {accuracy * 100:.2f}%")
print(f"Test Loss: {loss:.4f}")


<div style="background-color:#3CB371; color:white; padding:10px 20px; border-radius:8px; font-family: 'Arial', sans-serif; margin-top:15px; margin-bottom:10px; box-shadow: 0 3px 6px rgba(0,0,0,0.1);"><h3 style="margin:0; font-size:20px;">8.a) Confusion Matrix Visualization</h3><p style="color:white; font-size:15px; margin-top:5px; line-height:1.4;">The image displays a confusion matrix titled <strong>"Confusion Matrix - Before Fine-Tuning"</strong>, illustrating the performance of a plant disease classification model. Each cell shows how many instances of a true label were predicted as a specific class.</p><ul style="font-size:15px; line-height:1.5; color:white; margin-left:15px;"><li><strong>Diagonal cells</strong> — Represent <strong>True Positives (TP)</strong>, where predictions match actual disease labels.</li><li><strong>Off-diagonal cells</strong> — Indicate misclassifications, including <strong>False Positives (FP)</strong> and <strong>False Negatives (FN)</strong>.</li><li><strong>Color intensity</strong> — Reflects the number of samples per cell, with darker shades indicating higher counts.</li></ul><p style="color:white; font-size:15px; margin-top:5px; line-height:1.4;">This matrix helps pinpoint which plant disease categories are well predicted and which ones need further fine-tuning, guiding model optimization and dataset balancing.</p></div>

In [ ]:
val_image_generator.shuffle = False
pred_before = model_before.predict(val_image_generator)


In [ ]:
pred_after = model_after.predict(val_image_generator)

In [ ]:
true_labels = val_image_generator.classes

pred_before_classes = np.argmax(pred_before, axis=1)
pred_after_classes = np.argmax(pred_after, axis=1)

cm_before = confusion_matrix(true_labels, pred_before_classes)
cm_after = confusion_matrix(true_labels, pred_after_classes)

plt.figure(figsize=(12, 5))

# ---------- FIGURE 1 (Before) ----------
plt.figure(figsize=(6, 5))
sns.heatmap(cm_before, annot=True, fmt="d", cmap="Blues",
            xticklabels=class_names, yticklabels=class_names)
plt.title("Confusion Matrix - Before Fine-Tuning")
plt.xlabel("Predicted")
plt.ylabel("True")
plt.show()

# ---------- FIGURE 2 (After) ----------
plt.figure(figsize=(6, 5))
sns.heatmap(cm_after, annot=True, fmt="d", cmap="Greens",
             xticklabels=class_names, yticklabels=class_names)
plt.title("Confusion Matrix - After Fine-Tuning")
plt.xlabel("Predicted")
plt.ylabel("True")
plt.show()



<div style="background-color:#3CB371; color:white; padding:10px 20px; border-radius:8px; font-family: 'Arial', sans-serif; margin-top:15px; margin-bottom:10px; box-shadow: 0 3px 6px rgba(0,0,0,0.1);">
    <h3 style="margin:0; font-size:20px;">8.b) Classification Report</h3>
    <p style="color:white; font-size:15px; margin-top:5px; line-height:1.4;">
        The classification report summarizes the model’s performance on each plant disease class, both before and after fine-tuning.
        It provides metrics such as Precision, Recall, and F1-Score, which are crucial for understanding class-level performance.
    </p>
    <ul style="font-size:15px; line-height:1.5; color:white; margin-left:15px;">
        <li><strong>Precision</strong> — Proportion of correctly predicted instances among all predicted instances of a class.</li>
        <li><strong>Recall</strong> — Proportion of correctly predicted instances among all actual instances of a class.</li>
        <li><strong>F1-Score</strong> — Harmonic mean of Precision and Recall, giving a single metric for class performance.</li>
        <li><strong>Support</strong> — Number of true instances for each class in the dataset.</li>
    </ul>
    <p style="color:white; font-size:15px; margin-top:5px; line-height:1.4;">
        Comparing classification reports before and after fine-tuning helps identify which disease classes benefited most from fine-tuning and which classes still need improvement.
    </p>
</div>


In [ ]:
# ---- Classification Report ----
report = classification_report(true_labels, pred_before_classes, target_names=class_names)
print("\nClassification Report Before Fine-Tuning :")
print(report)

In [ ]:
# ---- Classification Report ----
report = classification_report(true_labels, pred_after_classes, target_names=class_names)
print("\nClassification Report After Fine-Tuning :")
print(report)

<div style="background-color:#3CB371; color:white; padding:10px 20px; border-radius:8px; font-family: 'Arial', sans-serif; margin-top:15px; margin-bottom:10px; box-shadow: 0 3px 6px rgba(0,0,0,0.1);">
    <h3 style="margin:0; font-size:20px;">8.c) Training Curves (Loss & Accuracy)</h3>
    <p style="color:white; font-size:15px; margin-top:5px; line-height:1.4;">
        The training curves visualize how the model’s loss and accuracy evolved over each epoch for both training and validation sets. 
        This helps to assess learning behavior and detect issues like overfitting or underfitting.
    </p>
    <ul style="font-size:15px; line-height:1.5; color:white; margin-left:15px;">
        <li><strong>Training Loss</strong> — Shows the decrease of loss on the training set.</li>
        <li><strong>Validation Loss</strong> — Indicates how well the model generalizes to unseen data.</li>
        <li><strong>Training Accuracy</strong> — Progression of correctly predicted samples during training.</li>
        <li><strong>Validation Accuracy</strong> — How accurately the model performs on validation data.</li>
    </ul>
    <p style="color:white; font-size:15px; margin-top:5px; line-height:1.4;">
        By examining these curves, we can understand the impact of fine-tuning on model performance and stability over epochs.
    </p>
</div>


In [ ]:
# Get the training history
h = hist.history

# Set plotting style
plt.style.use('ggplot')
plt.figure(figsize=(12, 8))
plt.title("Training vs Validation Accuracy and Loss Over Epochs (Before - FineTuning- )")

# Plot training and validation loss
plt.plot(h['loss'], c='red', label='Training Loss')
plt.plot(h['val_loss'], c='red', linestyle='--', label='Validation Loss')

# Plot training and validation accuracy
plt.plot(h['accuracy'], c='blue', label='Training Accuracy')
plt.plot(h['val_accuracy'], c='blue', linestyle='--', label='Validation Accuracy')

plt.xlabel("Number of Epochs")
plt.legend(loc='best')
plt.show()

In [ ]:
# Get the training history
h = fine_tune_history.history

# Set plotting style
plt.style.use('ggplot')
plt.figure(figsize=(12, 8))
plt.title("Training vs Validation Accuracy and Loss Over Epochs (After - FineTuning- )")


# Plot training and validation loss
plt.plot(h['loss'], c='red', label='Training Loss')
plt.plot(h['val_loss'], c='red', linestyle='--', label='Validation Loss')

# Plot training and validation accuracy
plt.plot(h['accuracy'], c='blue', label='Training Accuracy')
plt.plot(h['val_accuracy'], c='blue', linestyle='--', label='Validation Accuracy')

plt.xlabel("Number of Epochs")
plt.legend(loc='best')
plt.show()

<div style="background-color:#2E8B57; color:white; padding:10px 20px; border-radius:8px; font-family: 'Arial', sans-serif; margin-top:20px; margin-bottom:10px; box-shadow: 0 3px 6px rgba(0,0,0,0.1);">
    <h2 style="margin:0; font-size:24px;">9) Testing Images from Google</h2>
    <p style="color:white; font-size:16px; margin-top:5px; line-height:1.4;">
        To further evaluate the model's generalization capabilities, additional plant disease images were collected from Google and used as test samples.
        This step ensures the model is robust and can handle real-world images beyond the training dataset.
    </p>
    <ul style="font-size:16px; line-height:1.5; color:white; margin-left:15px;">
        <li><strong>Image Selection</strong> — Relevant images for each plant disease category were carefully selected to match dataset conditions.</li>
        <li><strong>Preprocessing</strong> — All images were resized, normalized, and preprocessed similarly to the training data for consistency.</li>
        <li><strong>Model Evaluation</strong> — Predictions were generated for these images and compared to actual disease labels to assess real-world performance.</li>
    </ul>
    <p style="color:white; font-size:16px; margin-top:5px; line-height:1.4;">
        Testing on external images provides insights into the model’s robustness, highlights potential limitations, and confirms whether fine-tuning improvements translate to unseen, real-world data.
    </p>
</div>


In [ ]:
def PredictImageFromPath(image_path,actual):

     # Load and preprocess the image
    img = load_img(image_path, target_size=(WIDTH,HEIGHT))
    img_array = preprocess_input(img_to_array(img))
    img_array_exp = np.expand_dims(img_array, axis=0) # Add batch dimension

    # Make prediction
    pred = model_after.predict(img_array_exp, verbose=0)
    class_index = np.argmax(pred, axis=1)[0]
    label = class_names[class_index]    

    # Display the image with predicted label
    plt.figure(figsize=(4, 4))
    plt.imshow(img)
    plt.title(f"Predicted: {label} \n Actual: {actual}")
    plt.axis('off')
    plt.show()
    

In [ ]:
PredictImageFromPath("/kaggle/input/last-test-images/Test_Images/bacterial pepper/Pepper-Bacterial-Leaf-Spot-Leaf.jpg","Pepper Bell Bacterial Spot")


In [ ]:
PredictImageFromPath("/kaggle/input/last-test-images/Test_Images/bacterial pepper/figure-2-tattered appearance on the affected leaves-893x595.jpg","Pepper Bell Bacterial Spot")


In [ ]:
PredictImageFromPath("/kaggle/input/last-test-images/Test_Images/Tomato Late blight/IMG_5813.jpg","Tomato Late Blight")


In [ ]:
PredictImageFromPath("/kaggle/input/last-test-images/Test_Images/Tomato Late blight/late_blight_tomato_leaf5x12001-1.jpg","Tomato Late Blight")


In [ ]:
PredictImageFromPath("/kaggle/input/last-test-images/Test_Images/tomato yellow virus/original.jpg","Tomato Yellow Leaf Curl Virus")


In [ ]:
PredictImageFromPath("/kaggle/input/last-test-images/Test_Images/tomato yellow virus/thumb.jpg","Tomato Yellow Leaf Virus")


In [ ]:
PredictImageFromPath("/kaggle/input/last-test-images/Test_Images/potato leaf healthy/The-three-Sample-leaves-of-potato-are-a-leaf-affected-by-Light-Blight-b-leaf.jpg","Potato Healthy")


In [ ]:
PredictImageFromPath("/kaggle/input/last-test-images/Test_Images/potato leaf healthy/original.jpg","Potato Healthy")


In [ ]:
PredictImageFromPath("/kaggle/input/last-test-images/Test_Images/septoria leaf spot/A-tomato-leaf-with-Septoria-leaf-spot-which-is-a-fungal-disease.png","Tomato Septoria  Leaf")


In [ ]:
PredictImageFromPath("/kaggle/input/last-test-images/Test_Images/septoria leaf spot/tomato-septoria-leaf-spot-cropped.jpg","Tomato Septoria  Leaf")
